In [1]:
!which python

/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/bin/python


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
stack_overflow_dataset = "Mxode/StackOverflow-QA-C-Language-5k"

# Fine-tuned model
new_model = "llama-2-7b-chat-stack-overflow"

In [4]:
dataset = load_dataset(stack_overflow_dataset, split="train")

In [5]:
# dataset currently has 'question' and 'answer' columns. We need to combine them into a single column named 'text'
dataset = dataset.map(
    lambda examples: {
        "text": "<s>[INST]" + examples["question"] + "[\INST] " + examples["answer"] + "</s>",
    }
)

In [8]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [9]:
#!pip install accelerate

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    #low_cpu_mem_usage=True,
    # quantization_config=quant_config,
    # device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [12]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [13]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/5151 [00:00<?, ? examples/s]

/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


('llama-2-7b-chat-stack-overflow/tokenizer_config.json',
 'llama-2-7b-chat-stack-overflow/special_tokens_map.json',
 'llama-2-7b-chat-stack-overflow/tokenizer.json')

Now lets compare asking 3 questions to our fine tuned model vs our base model

First the fine tuned model

In [16]:
#logging.set_verbosity(logging.CRITICAL)

prompt = "How do I loop through an array in C?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How do I loop through an array in C? [/INST]  In C, you can loop through an array using a `for` loop or a `while` loop. Here are the basic syntaxes for each:

For loop:
```
for (int i = 0; i < array_size; i++) {
    // loop body
}
```
In this syntax, `array_size` is the number of elements in the array, and `i` is the index of the current element.

While loop:
```
int i = 0;
while (i < array_size) {
    // loop body
    i++;
}
```
In this syntax, `i` is initialized to 0, and the loop continues until `i` is greater than or equal to `array_size`.

You can also use a `for_each` function in


In [17]:
prompt = "What does the 'sizeof' operator do in C?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What does the 'sizeof' operator do in C? [/INST]  In C, the `sizeof` operator is used to determine the size of a variable or an object in memory.

The `sizeof` operator takes a single argument, which can be a variable, a type, or a constant expression. The operator returns the size of the object or variable in bytes.

Here are some examples of how to use the `sizeof` operator in C:

1. To get the size of a variable:
```
int x = 5;
printf("The size of x is %zu\n", sizeof(x)); // Output: The size of x is 4
```
In this example, `x` is an `int` variable with a size of 4 bytes.

2. To get the size of a type:
```
struct Person {
    int age;
    char name[


In [18]:
prompt = "How to fix a segmentation fault in C?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How to fix a segmentation fault in C? [/INST]  A segmentation fault, also known as a segmentation violation or SEGMENTATION_FAULT, is a type of error that occurs when a program attempts to access memory outside of its allocated boundaries. Here are some steps to help you fix a segmentation fault in C:

1. Identify the location of the fault: Use a debugger or print the backtrace to determine where the segmentation fault occurred. This will help you identify the line of code that is causing the error.
2. Review the code: Look at the code that caused the segmentation fault and make sure that you are not trying to access memory outside of the allocated boundaries. Check if you are using an index that is out of range or if you are trying to access a null pointer.
3. Check for null pointers: Make sure that you are not trying to access a null pointer. A null


Now the base model

In [19]:

prompt = "How do I loop through an array in C?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<s>[INST] How do I loop through an array in C? [/INST]  In C, you can loop through an array using a `for` loop or a `while` loop. Here are the basic syntaxes for each:

For loop:
```
for (int i = 0; i < array_size; i++) {
    // loop body
}
```
In this syntax, `array_size` is the number of elements in the array, and `i` is the index of the current element.

While loop:
```
int i = 0;
while (i < array_size) {
    // loop body
    i++;
}
```
In this syntax, `i` is initialized to 0, and the loop continues until `i` is greater than or equal to `array_size`.

You can also use a `for_each` function in


In [21]:
prompt = "What does the 'sizeof' operator do in C?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<s>[INST] What does the 'sizeof' operator do in C? [/INST]  In C, the `sizeof` operator is used to determine the size of a variable or an object in memory.

The `sizeof` operator takes a single argument, which can be a variable, a type, or a constant expression. The operator returns the size of the object or variable in bytes.

Here are some examples of how to use the `sizeof` operator in C:

1. To get the size of a variable:
```
int x = 5;
printf("The size of x is %zu\n", sizeof(x)); // Output: The size of x is 4
```
In this example, `x` is an integer variable with a size of 4 bytes.

2. To get the size of a type:
```
struct Person {
    int age;
    char name[20


In [22]:
prompt = "How to fix a segmentation fault in C?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<s>[INST] How to fix a segmentation fault in C? [/INST]  A segmentation fault, also known as a segmentation violation or SEGMENTATION_FAULT, is a type of error that occurs when a program attempts to access memory outside of its allocated boundaries. Here are some steps to help you fix a segmentation fault in C:

1. Identify the location of the fault: Use a debugger or print the backtrace to determine where the segmentation fault occurred. This will help you identify the line of code that is causing the error.
2. Review the code: Look at the code that caused the segmentation fault and make sure that you are not trying to access memory outside of the allocated boundaries. Check if you are using an index that is out of range or if you are trying to access a null pointer.
3. Check for null pointers: Make sure that you are not trying to access a null pointer. A null
